In [1]:
# !pip install einops

In [2]:
from transformers import T5Tokenizer, FlaxMT5ForConditionalGeneration
import jax
from flax import jax_utils
from flax.training.common_utils import shard
import einops

/opt/conda/envs/hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-18 13:02:18.689099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 13:02:19.581885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-01-18 13:02:19.582000: W tensorflow/compiler/xla/stream_ex

In [3]:
# only using 1 GPU, hence only 1 JAX device available; on a TPU v3-8 you should expect this to return 8 (since we have 8 TPU cores)
print(jax.device_count())

1


In [4]:
model, params = FlaxMT5ForConditionalGeneration.from_pretrained("google/mt5-small", _do_init=False)
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

In [5]:
input_context = ["test string" for i in range(8)]  # change to the prompt(s) you want to test
tokenized_context = tokenizer(input_context, return_tensors="np")

In [6]:
# shard our data across accelerator devices (i.e. data parallelism, [seq_1, seq_2, ..., seq_8] -> TPU_1: [seq_1], TPU_2: [seq_2], ... , TPU_8: [seq_8])
input_ids = shard(tokenized_context.input_ids)
attention_mask = shard(tokenized_context.attention_mask)

# only using 1 GPU, so effectively no data parallelisation across devices; on TPU the data will be sharded across your 8 devices so this should be shape `(8, 1, 3)`
input_ids.shape

(1, 8, 3)

In [7]:
# copy (replicate) the params across your TPU devices
params = jax_utils.replicate(params)

In [8]:
# pmap generate (like jit, but replicated across our JAX devices)
pmap_generate = jax.pmap(model.generate, "batch")

In [15]:
outputs = pmap_generate(input_ids, attention_mask=attention_mask, params=params).sequences

In [16]:
# on TPU the returned sequences will be sharded across your 8 devices, so this should be shape `(8, 1, 20)`
outputs.shape

(1, 8, 20)

In [17]:
# helper function to 'unshard' our returned sequences
unshard = lambda x: einops.rearrange(x, 'd b ... -> (d b) ...')

In [18]:
outputs = unshard(outputs)
outputs.shape

(8, 20)

In [19]:
# we get garbage outputs, but this is understandable since mT5 is not fine-tuned on a downstream task (see https://github.com/huggingface/transformers/issues/8704), replace with your fine-tuned model for applicable results
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['<extra_id_0>',
 '<extra_id_0>',
 '<extra_id_0>',
 '<extra_id_0>',
 '<extra_id_0>',
 '<extra_id_0>',
 '<extra_id_0>',
 '<extra_id_0>']